In [18]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge

# Distinct Functions

In [2]:
def distinct_n(sentences, n):
    all_grams = []
    for sentence in sentences:
        tokens = word_tokenize(sentence.lower())
        if n > len(tokens):
            continue
        else:
            grams = [tokens[i:(i+n)] for i in range(len(tokens) - n + 1)]
            grams = [' '.join(gram) for gram in grams]
            all_grams += grams
    return len(set(all_grams)) / len(all_grams)

# Calculate Scores

In [3]:
df = pd.read_csv('../data/test/result/all/all_result.csv')
df

,post_id,post,most_sim_post_id,most_sim_post_emo,most_sim_post,cos_sim,highest_degree_id,highest_degree,random_id,random,...,follow_id,follow_emo,follow,moel,mime,cem,blender,meed,groundtruth_id,groundtruth
0,r2w43k,Do you ever feel like everyone else has it tog...,79pxlq,jealous,I feel like everyone else has their life more ...,0.761634,dp3wsyf,Good luck my dude,dp3vg2a,I feel like you have a more together life than...,...,dp3vowt,jealous,This lack of self acceptance I think also feed...,that is great ! i wish you the best of luck !,i am sorry to hear that . i am sorry to hear t...,i am sorry to hear that .,"yes , i feel like i ' m the only one that does...",Not really. Why do you say that?,hm70wwb\nhm7608z,I hope this kinda helps.\nOne of the gifts of ...
1,r2qqyz,I've finally figured out the answer to the que...,dce0wm,questioning,"So, what’re your favorite movies?",0.681865,f27kx8t,Hot Rod.,f27oy5w,My favorite is Tangled,...,f27kx8t,questioning,Hot Rod.,that sounds like a lot of fun !,that is a good thing to do . what kind of musi...,"me too , i love watching tv shows myself .",i love role playing games . it ' s a form of r...,What kind of movies do you like?,hm61y7k\nhm63dh1,"Man, I wouldn't know where to get started.\nHa..."
2,r2doqc,I cooked the entree for my Friendsgiving for t...,cfmhp5,proud,I cooked a real meal last night for the first ...,0.770251,eubc9si,Congratulations!,eub2smt,Homemade food always tastes better as well.,...,eucjcz4,proud,"Professional chef here,.",that sounds like a lot of fun !,why did you do that ?,what kind of cake was it ?,that ' s so cool ! what did you do with it ? d...,That's awesome! What kind of food was it?,hm49qhc\nhm4n8f9\nhm4xknl,Happy Thanksgiving :)\nI miss celebrating Than...
3,r2cedp,"At my job where I work with kids 5-13, I start...",omtovj,jealous,Last friday I found out my job don't mind men ...,0.583775,h5nqfyp,Good luck!,h5o2hge,It makes the nail polish easier to scrape off.,...,h5rkkub,jealous,Also it's a great way to bond with a female fr...,that is great ! i am sorry you have to deal wi...,that is a shame . i am sorry to hear that . ho...,that is nice of them,that ' s pretty cool . i ' ve always wanted to...,That sounds like a lot of work. What kind of w...,hm6enwq,keep painting and being awesome 😎
4,r1zcsu,Having enough food and water.,msfr6b,confident,Working out eating right.,0.541680,gusf87s,"I'm not perfect, but I think it's important to...",gusb66v,Well i personally believe that true friends un...,...,gusb66v,confident,Well i personally believe that true friends un...,that sounds like a good plan .,that is a good thing to do,what kind of food is it ?,have you ever been in a situation where you co...,What kind of food do you have?,hm1m52g\nhm1mnox,Lol.\nHaving enough food and water.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15207,hit:12318_conv:24637_uttr:1,"I don't like scammy sales people, nothing good...",87s0d7,trusting,Beware of crooked salesman that come to your door,0.621139,dwf2g1h,Aint no one home.,dwf39we,I really worry about elderly people in these k...,...,dwf2g1h,lonely,Aint no one home.,why is that ?,i am sorry to hear that . what happened ?,oh no ! what happened ?,"yoga is a physical , mental , and spiritual pr...",Why is that?,hit:12318_conv:24637_uttr:2,"I agree, were you recently screwed by one?"
15208,hit:12384_conv:24768_uttr:1,"I almost got caught stealing, I felt pretty bad",hit:12314_conv:24629_uttr:1,guilty,I stole from my friend and i feel pretty bad a...,0.776447,hit:12314_conv:24629_uttr:2,Why did you steal stuff?,hit:12314_conv:24629_uttr:2,Why did you steal stuff?,...,hit:12314_conv:24629_uttr:2,questioning,Why did you steal stuff?,why did you do that ?,that is terrible . what did you do ?,what happened ?,"oh no , why did you feel bad ? india is a coun...",Why did you do that?,hit:12384_conv:24768_uttr:2,Ouch!
15209,hit:12390_conv:24781_uttr:1,Some random dude walked into my room while I w...

In [4]:
score_dict = {}
score_cols = ['model', 'dist_1', 'dist_2', 'dist_3', 'dist_4',
              'bleu_1', 'bleu_2', 'bleu_3', 'bleu_4', 'meteor', 'rouge']
for col in score_cols:
    score_dict[col] = []

In [5]:
models = ['moel', 'mime', 'cem', 'blender', 'meed',
          'highest_degree', 'random', 'intent', 'follow']
score_dict['model'] = models

## Distinct Scores

In [6]:
for n in range(1, 5):
    scores = []
    print('Distinct-{}'.format(n))
    for model in tqdm(models):
        score = distinct_n(df[model].tolist(), n)
        scores.append(score)
    score_dict['dist_{}'.format(n)] = scores

Distinct-1


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.17it/s]


Distinct-2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.17it/s]


Distinct-3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.17it/s]


Distinct-4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:07<00:00,  1.17it/s]


## BLEU Scores

In [7]:
all_refs = []
for i in tqdm(range(df.shape[0])):
    refs = [word_tokenize(x.lower()) for x in df.iloc[i]['groundtruth'].split('\n')]
    all_refs.append(refs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15212/15212 [00:04<00:00, 3176.64it/s]


In [8]:
weights = [(1, 0, 0, 0), (1/2, 1/2, 0, 0), (1/3, 1/3, 1/3, 0), (1/4, 1/4, 1/4, 1/4)]

In [9]:
for n in range(1, 5):
    print('BLEU-{}'.format(n))
    model_scores = []
    for model in tqdm(models):
        scores = []
        for i in range(df.shape[0]):
            hyp = word_tokenize(df.iloc[i][model].lower())
            score = sentence_bleu(all_refs[i], hyp, weights = weights[n - 1])
            scores.append(score)
        model_scores.append(np.mean(scores))
    score_dict['bleu_{}'.format(n)] = model_scores

BLEU-1


  0%|                                                                                                                                     | 0/9 [00:00<?, ?it/s]/Users/xieyubo/venvs/ml/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/xieyubo/venvs/ml/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/xieyubo/venvs/ml/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram ove

BLEU-2


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.63s/it]


BLEU-3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.61s/it]


BLEU-4


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:32<00:00,  3.62s/it]


In [10]:
score_dict['bleu_1']

[0.22962240367981723,
 0.2392414106286214,
 0.19053659399481754,
 0.20835261019004123,
 0.2059950317452723,
 0.20748700671023526,
 0.20775149306804616,
 0.20605988334109093,
 0.20600520853023646]

In [11]:
score_dict['bleu_2']

[0.023233389007869403,
 0.021474896332854675,
 0.01837375057841722,
 0.03828309327640678,
 0.02999384618707307,
 0.05282980891612178,
 0.047504186164122185,
 0.0481058884797783,
 0.04638499685192393]

In [12]:
score_dict['bleu_3']

[0.00517727237918219,
 0.0031703096726403857,
 0.003803109513851687,
 0.0086375463516098,
 0.007385021699928539,
 0.02528003986877344,
 0.027072778204586957,
 0.026316328353371804,
 0.026949049692578296]

In [13]:
score_dict['bleu_4']

[0.0016468522349033315,
 0.0011486439484681898,
 0.0012886757521415678,
 0.0022249325454760677,
 0.00245143154739919,
 0.01809016045799307,
 0.02233772056117891,
 0.02044847518192572,
 0.02227162625992655]

## METEOR Score

In [20]:
model_scores = []
for model in tqdm(models):
    scores = []
    for i in range(df.shape[0]):
        hyp = word_tokenize(df.iloc[i][model].lower())
        score = meteor_score(all_refs[i], hyp)
        scores.append(score)
    model_scores.append(np.mean(scores))
score_dict['meteor'] = model_scores

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [03:32<00:00, 23.61s/it]


In [21]:
score_dict['meteor']

[0.10984214652892174,
 0.12976614109791895,
 0.08616224621397131,
 0.16154508379339846,
 0.09724203856353157,
 0.12739330023479353,
 0.12936657544715413,
 0.12568719773084935,
 0.12746072009102236]

## ROUGE Score

In [26]:
rouge = Rouge()

In [28]:
def rouge_score(refs, hyp):
    scores_1 = []
    scores_2 = []
    scores_l = []
    for ref in refs:
        score = rouge.get_scores(hyp, ref)
        scores_1.append(score[0]['rouge-1']['f'])
        scores_2.append(score[0]['rouge-2']['f'])
        scores_l.append(score[0]['rouge-l']['f'])
    return np.mean(scores_1), np.mean(scores_2), np.mean(scores_l)

In [30]:
model_rouge_1_scores = []
model_rouge_2_scores = []
model_rouge_l_scores = []
for model in tqdm(models):
    rouge_1_scores = []
    rouge_2_scores = []
    rouge_l_scores = []
    for i in range(df.shape[0]):
        hyp = df.iloc[i][model].lower()
        refs = [x.lower() for x in df.iloc[i]['groundtruth'].split('\n')]
        score_1, score_2, score_l = rouge_score(refs, hyp)
        rouge_1_scores.append(score_1)
        rouge_2_scores.append(score_2)
        rouge_l_scores.append(score_l)
    model_rouge_1_scores.append(np.mean(rouge_1_scores))
    model_rouge_2_scores.append(np.mean(rouge_2_scores))
    model_rouge_l_scores.append(np.mean(rouge_l_scores))
score_dict['rouge_1'] = model_rouge_1_scores
score_dict['rouge_2'] = model_rouge_2_scores
score_dict['rouge_l'] = model_rouge_l_scores

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:09<00:00,  7.72s/it]


# Generate CSV

In [31]:
score_df = pd.DataFrame(score_dict)
score_df

,model,dist_1,dist_2,dist_3,dist_4,bleu_1,bleu_2,bleu_3,bleu_4,meteor,rouge_1,rouge_2,rouge_l
0,moel,0.002095,0.009233,0.019055,0.031880,0.229622,0.023233,0.005177,0.001647,0.109842,0.057756,0.004015,0.054445
1,mime,0.001138,0.004178,0.007350,0.011755,0.239241,0.021475,0.003170,0.001149,0.129766,0.068163,0.003373,0.064858
2,cem,0.002084,0.008022,0.015074,0.024662,0.190537,0.018374,0.003803,0.001289,0.086162,0.042607,0.003244,0.040188
3,blender,0.015092,0.087334,0.177834,0.260219,0.208353,0.038283,0.008638,0.002225,0.161545,0.078195,0.005354,0.072076
4,meed,0.014091,0.068132,0.131405,0.202345,0.205995,0.029994,0.007385,0.002451,0.097242,0.044996,0.004695,0.043213
5,highest_degree,0.070304,0.428573,0.736314,0.847439,0.207487,0.052830,0.025280,0.018090,0.127393,0.051756,0.012443,0.049169
6,random,0.070176,0.452708,0.791475,0.910602,0.207751,0.047504,0.027073,0.022338,0.129367,0.055547,0.011024,0.051883
7,intent,0.070317,0.438091,0.759237,0.877826,0.206060,0.048106,0.026316,0.020448,0.125687,0.052904,0.012085,0.049865
8,follow,0.069572,0.443266,0.769506,0.880802,0.206005,0.046385,0.026949,0.022272,0.127461,0.054464,0.010959,0.050883


In [32]:
score_df.to_csv('../data/test/result/all/auto_eval_result.csv', index = False)